# KL divergence NMF (KLNMF) SCIPI

## Example: real data

In [1]:
# We will use waving trees
# need to download the data and set relative/absolute paths

### Load Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from PIL import Image

### Load Sourcecode

In [3]:
import sys, os
sys.path.append(os.path.join(os.path.dirname(sys.argv[1]), '..', 'src'))
import klnmf
import importlib
importlib.reload(klnmf)

<module 'klnmf' from 'C:\\Users\\bions\\Desktop\\git\\SCIPI-JMLR\\notebooks\\..\\src\\klnmf.py'>

### Set Random Seed

In [4]:
today_num = int(pd.Timestamp.today().date().strftime("%Y%m%d"))
offset = 0
print(f"our seed is {today_num + offset}")
np.random.seed(today_num + offset)

our seed is 20231019


### Read data

In [5]:
# you need to remove the first 3 lines from docword.kos
# those lines have this metadata info
n = 120 * 160
m = 287

In [6]:
from PIL import Image
our_dtype = 'float32'

### Set Size

In [7]:
k = 20
our_dtype = 'float32'

files = [f for f in os.listdir("../../wavingtrees") if f[-4:] == '.bmp']
X = np.zeros((120 * 160, 287))
for i in range(len(files)):
    X[:,i] = np.asarray(Image.open("../../wavingtrees/" + files[i]).convert('L')).reshape(-1).astype(our_dtype)

V_orig = X[:3000, :]
V_orig = V_orig / V_orig.sum().sum() * k
V_orig

array([[8.45581667e-06, 8.45581667e-06, 8.45581667e-06, ...,
        8.76330091e-06, 8.60955879e-06, 8.60955879e-06],
       [8.60955879e-06, 8.60955879e-06, 8.60955879e-06, ...,
        8.60955879e-06, 8.60955879e-06, 8.60955879e-06],
       [8.45581667e-06, 8.76330091e-06, 8.76330091e-06, ...,
        8.60955879e-06, 8.76330091e-06, 8.76330091e-06],
       ...,
       [2.30613182e-06, 1.53742121e-06, 1.84490546e-06, ...,
        2.61361606e-06, 2.45987394e-06, 2.45987394e-06],
       [9.07078516e-06, 8.30207455e-06, 8.30207455e-06, ...,
        7.84084819e-06, 8.45581667e-06, 8.76330091e-06],
       [6.45716909e-06, 7.37962182e-06, 6.61091122e-06, ...,
        1.07619485e-05, 1.04544642e-05, 1.01469800e-05]])

In [8]:
# We resize the scale of V_orig.
# This is not requirede but to ease the objective calculation.
# V_orig is our target matrix to be decomposed

In [9]:
print(f"size of V: {V_orig.shape}")

size of V: (3000, 287)


### Our matrix to be decomposed

In [10]:
V_orig[:4, :4]

array([[8.45581667e-06, 8.45581667e-06, 8.45581667e-06, 8.60955879e-06],
       [8.60955879e-06, 8.60955879e-06, 8.60955879e-06, 8.60955879e-06],
       [8.45581667e-06, 8.76330091e-06, 8.76330091e-06, 8.60955879e-06],
       [8.60955879e-06, 8.76330091e-06, 8.76330091e-06, 8.60955879e-06]])

### Initialization

In [11]:
W_mat, H_mat, A_mat = klnmf.init_klnmf(V_orig, k, seed = 1, our_dtype = our_dtype)

In [12]:
# we will use the same initialization for all the method
# the above function `init_klnmf` provies 1-step MU initialization from random matrix
# please see the manuscript for details

### Run Methods

In [13]:
# note that we need a sparse version of each method
# we need to compute this
# A = V / (W @ H + eps)
# However W @ H can be 0 and regardless of V we have NaNs
# so we set eps to be nonzero

#### MU (Multiplicatsive Updates)

In [14]:
res_mu = klnmf.run_mu(V_orig, k, num_iter = 1000, num_print = 50)

init: obj 41.18533858260692
round 50: obj 41.083559468469765
round 100: obj 41.034406290892186
round 150: obj 41.01326375645623
round 200: obj 40.99983925067989
round 250: obj 40.99178036166448
round 300: obj 40.986727741584886
round 350: obj 40.983393118766706
round 400: obj 40.981112600152244
round 450: obj 40.979482512999574
round 500: obj 40.97826645888894
round 550: obj 40.977327125226644
round 600: obj 40.97657800912874
round 650: obj 40.97596428561111
round 700: obj 40.97545110420362
round 750: obj 40.97501523622918
round 800: obj 40.97463947531677
round 850: obj 40.97431176467013
round 900: obj 40.97402440254642
round 950: obj 40.97377084148396
round 1000: obj 40.9735448572233


In [15]:
res_mu_with_normalize = klnmf.run_mu_with_normalize(V_orig, k, num_iter = 1000, num_print = 50)

init: obj 41.18533858260692
round 50: obj 41.08410197867809
round 100: obj 41.03458476462467
round 150: obj 41.01342018632649
round 200: obj 40.99999289928605
round 250: obj 40.99192477018798
round 300: obj 40.98688292425212
round 350: obj 40.98355739716154
round 400: obj 40.98128169635627
round 450: obj 40.97965667512553
round 500: obj 40.97844628510202
round 550: obj 40.97751144059887
round 600: obj 40.976765455402784
round 650: obj 40.976153360106665
round 700: obj 40.97564164789364
round 750: obj 40.97520690437274
round 800: obj 40.974831890421825
round 850: obj 40.97450565499373
round 900: obj 40.97421972777031
round 950: obj 40.973966127783534
round 1000: obj 40.97373804292444


In [16]:
# run_mu is running mu without rescaling every round
# run_mu_with_normalize is running mu with rescaling every round
# they are visually the same
# however one is a little slower due to rescaling
# the other is a little numerically instable
# however for this example they are nearly identical

#### SCIPI (Scale Invariant Power Iteration)

In [17]:
# we have advanced version of SCIPI
# please see the paper for advanced methods
# but here we just run a plane method

In [18]:
res_scipi = klnmf.run_scipi(V_orig, k, num_iter = 1000, num_print = 50)

init: obj 41.18533858260692
round 50: obj 41.03500884008213
round 100: obj 41.000114276841174
round 150: obj 40.98694400320204
round 200: obj 40.98133938666514
round 250: obj 40.97850264998479
round 300: obj 40.976818888974115
round 350: obj 40.97569265991618
round 400: obj 40.97488167505706
round 450: obj 40.97426948047415
round 500: obj 40.97378912638583
round 550: obj 40.97339588798353
round 600: obj 40.973067403626644
round 650: obj 40.97278502206613
round 700: obj 40.972533137172206
round 750: obj 40.97231345917747
round 800: obj 40.972121252023555
round 850: obj 40.97195096661652
round 900: obj 40.971796429121156
round 950: obj 40.971656387707114
round 1000: obj 40.971528518672194


In [19]:
# just for this example, here's the advanced scipi approach

In [20]:
res_scipi_acc = klnmf.run_scipi_acc(V_orig, k, num_inner = 2, intercept = 0.1, num_iter = 1000, num_print = 50)

init: obj 41.18533858260692
round 50: obj 41.00469279967758
round 100: obj 40.98291080094088
round 150: obj 40.977593307711665
round 200: obj 40.975371906724874
round 250: obj 40.97413414240629
round 300: obj 40.97334204779822
round 350: obj 40.97277390604017
round 400: obj 40.9723329033805
round 450: obj 40.97199082857274
round 500: obj 40.971708734580844
round 550: obj 40.971473024372486
round 600: obj 40.97127360584326
round 650: obj 40.97110299810493
round 700: obj 40.9709509401032
round 750: obj 40.970816837239035
round 800: obj 40.97070466248694
round 850: obj 40.97060952261787
round 900: obj 40.97052829503507
round 950: obj 40.97045214803494
round 1000: obj 40.97038257210512


#### PGD (Projected Gradient Descent)

In [22]:
# lots of papers about projection onto the simplex
# e.g.
# https://arxiv.org/pdf/1101.6081.pdf
# https://math.stackexchange.com/questions/3778014/matlab-python-euclidean-projection-on-the-simplex-why-is-my-code-wrong
# https://stanford.edu/~jduchi/projects/DuchiShSiCh08.html
# https://link.springer.com/article/10.1007/s10107-015-0946-6
# https://gist.github.com/mblondel/6f3b7aaad90606b98f71

In [23]:
# we choose the fastest one here among the above

In [24]:
res_pgd = klnmf.run_pgd(V_orig, k, stepsize = 2.0, num_iter = 1000, num_print = 50)

init: obj 41.18533858260692
round 50: obj 41.0349168773927
round 100: obj 41.00000666001572
round 150: obj 40.98680904255436
round 200: obj 40.9811850882095
round 250: obj 40.97833377160699
round 300: obj 40.97664030927893
round 350: obj 40.97551080708172
round 400: obj 40.97469791896701
round 450: obj 40.974083225118456
round 500: obj 40.97360481528682
round 550: obj 40.97321802767355
round 600: obj 40.972893817348094
round 650: obj 40.97262000272156
round 700: obj 40.97238337933514
round 750: obj 40.97217399111044
round 800: obj 40.97199343808836
round 850: obj 40.97183294331585
round 900: obj 40.971691980750734
round 950: obj 40.97156584509452
round 1000: obj 40.971450523350484


In [25]:
# stepsize 2.0 seems faster in early rounds but slower in later rounds

In [26]:
res_pgd2 = klnmf.run_pgd(V_orig, k, stepsize = 1.5, num_iter = 1000, num_print = 50)

init: obj 41.18533858260692
round 50: obj 41.05190413527297
round 100: obj 41.013362252394806
round 150: obj 40.99532502881657
round 200: obj 40.98672478763575
round 250: obj 40.982143871134305
round 300: obj 40.97947364800794
round 350: obj 40.97775727823686
round 400: obj 40.97656518829914
round 450: obj 40.97568287521526
round 500: obj 40.97500079698912
round 550: obj 40.974455507711
round 600: obj 40.974010023263695
round 650: obj 40.97364079190787
round 700: obj 40.97332842284599
round 750: obj 40.97305785905876
round 800: obj 40.97281900853292
round 850: obj 40.97260872122694
round 900: obj 40.97242156501554
round 950: obj 40.97224980061771
round 1000: obj 40.97209624266008


In [27]:
# stepsize > 2.1 will output NaN error because stepsize is too large

In [28]:
res_pgd3 = klnmf.run_pgd(V_orig, k, stepsize = 2.1, num_iter = 50, num_print = 50)

init: obj 41.18533858260692
round 50: obj nan


C:\Users\bions\Desktop\git\SCIPI-JMLR\notebooks\..\src\klnmf.py:281: RuntimeWarning: divide by zero encountered in divide
  A = V / (W @ H)
C:\Users\bions\Desktop\git\SCIPI-JMLR\notebooks\..\src\klnmf.py:282: RuntimeWarning: invalid value encountered in matmul
  W = W + (A @ H.T - H.sum(axis = 1, keepdims = True).T) * W / H.sum(axis = 1, keepdims = True).T * stepsize


In [29]:
# too slow
# and too bad
# this type of problems are not good with line search
# didn't run below

In [30]:
res_pgd_with_linesearch = klnmf.run_pgd_with_linesearch_for_sparse(V_orig, k, num_iter = 1000, num_print = 50)

init: obj 41.17668414246275
round 50: obj 41.1775326644039
round 100: obj 41.17825171865389
round 150: obj 41.17884248032172
round 200: obj 41.1793223900724
round 250: obj 41.17970655934998
round 300: obj 41.18000807814909
round 350: obj 41.18023828267752
round 400: obj 41.180406988032004
round 450: obj 41.18052269037567
round 500: obj 41.180592742546274
round 550: obj 41.18062350653141
round 600: obj 41.180620485812874
round 650: obj 41.18058844020157
round 700: obj 41.18053148544967
round 750: obj 41.18045317963385
round 800: obj 41.180356598046515
round 850: obj 41.18024439810787
round 900: obj 41.18011887561521
round 950: obj 41.17998201347474
round 1000: obj 41.1798355239116
